In [ ]:
# ==============================
# AI StudyBuddy - Full Advanced Version (Fixed)
# ==============================

!pip install gradio transformers sentencepiece pypdf2 python-docx python-pptx

import gradio as gr
import random, os, time
from datetime import datetime
from transformers import pipeline
import PyPDF2, docx, pptx

# -----------------------------
# 📂 File Reader
# -----------------------------
def read_file(file):
    if file.name.endswith(".pdf"):
        reader = PyPDF2.PdfReader(file.name)
        text = " ".join([page.extract_text() for page in reader.pages if page.extract_text()])
    elif file.name.endswith(".docx"):
        doc = docx.Document(file.name)
        text = " ".join([p.text for p in doc.paragraphs])
    elif file.name.endswith(".pptx"):
        pres = pptx.Presentation(file.name)
        text = " ".join([shape.text for slide in pres.slides for shape in slide.shapes if hasattr(shape,"text")])
    else:
        text = file.read().decode("utf-8")
    return text

# -----------------------------
# 📝 AI Models
# -----------------------------
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
qa_model = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
motivations = [
    "🚀 Keep pushing, success is near!",
    "📘 One chapter at a time, you’ll get there.",
    "💡 Hard work always pays off.",
    "🔥 Stay focused, your goals need you!"
]

# -----------------------------
# 🧩 Features
# -----------------------------
notes_store = []
progress_tracker = {"summaries":0, "quizzes":0, "notes":0, "flashcards":0}

def generate_summary(file):
    text = read_file(file)
    if not text.strip():
        return "⚠️ Could not extract text."
    chunks = [text[i:i+1000] for i in range(0,len(text),1000)]
    summary = " ".join([summarizer(c, max_length=100, min_length=30, do_sample=False)[0]['summary_text'] for c in chunks[:3]])
    progress_tracker["summaries"] += 1
    return summary

def generate_quiz(file):
    text = read_file(file).split(".")
    questions = []
    for idx, sent in enumerate(text[:5]):
        if len(sent.split()) > 6:
            q = f"❓ Q{idx+1}: {sent.strip()}?"
            opts = ["A. " + " ".join(sent.split()[:3]),
                    "B. " + " ".join(sent.split()[-3:]),
                    "C. Important keyword",
                    "D. None of the above"]
            correct = random.choice(opts)
            questions.append(f"{q}\nOptions: {', '.join(opts)}\n✅ Answer: {correct}\n")
    progress_tracker["quizzes"] += 1
    return "\n".join(questions)

def flashcards(file):
    text = read_file(file).split(".")
    cards = []
    for idx, sent in enumerate(text[:5]):
        cards.append({"question": f"{sent.strip()}?", "answer": "Review key concepts."})
    progress_tracker["flashcards"] += 1
    return cards

def ask_question(file, question):
    text = read_file(file)
    return qa_model(question=question, context=text)["answer"]

def save_notes(note):
    notes_store.append({"note": note, "time": datetime.now().strftime("%H:%M:%S")})
    progress_tracker["notes"] += 1
    return "\n".join([f"- {n['note']} ({n['time']})" for n in notes_store])

def clear_notes():
    notes_store.clear()
    return "🗑️ Notes cleared!"

def download_notes():
    content = "\n".join([f"- {n['note']} ({n['time']})" for n in notes_store])
    with open("MyNotes.txt","w") as f:
        f.write(content)
    return "MyNotes.txt"

def motivation_popup():
    return random.choice(motivations)

def exam_predictor(file):
    text = read_file(file).split(".")
    important = [s for s in text if len(s.split())>8]
    return "\n".join([f"📌 {s.strip()}" for s in important[:5]])

def track_progress():
    return "\n".join([f"{k}: {v}" for k,v in progress_tracker.items()])

# -----------------------------
# 👤 Simple Login
# -----------------------------
def login(user, pwd):
    if user and pwd:  # anyone can login
        return gr.update(visible=True), "✅ Login successful!"
    else:
        return gr.update(visible=False), "❌ Enter username/password"

# -----------------------------
# 🎨 Gradio UI
# -----------------------------
with gr.Blocks() as demo:
    gr.Markdown("# 📘 AI StudyBuddy\nYour smart learning assistant 🚀")

    with gr.Tab("🔐 Login"):
        user = gr.Textbox(label="Username")
        pwd = gr.Textbox(label="Password", type="password")
        login_btn = gr.Button("Login")
        login_status = gr.Label()
        hidden = gr.Group(visible=False)

    with hidden:
        with gr.Tab("📂 Upload & Summary"):
            file = gr.File(label="Upload Study File", file_types=[".pdf",".docx",".pptx",".txt"])
            summary_btn = gr.Button("Generate Summary")
            summary_out = gr.Textbox(label="Summary", lines=8)

        with gr.Tab("📝 Ask AI"):
            ques = gr.Textbox(label="Enter a Question")
            ans_btn = gr.Button("Get Answer")
            ans_out = gr.Textbox(label="Answer", lines=3)

        with gr.Tab("🎯 Quiz"):
            quiz_btn = gr.Button("Generate Quiz")
            quiz_out = gr.Textbox(label="Quiz Questions", lines=12)

        with gr.Tab("📇 Flashcards"):
            fc_btn = gr.Button("Generate Flashcards")
            fc_out = gr.JSON(label="Flashcards")

        with gr.Tab("🗒️ Notes"):
            note_in = gr.Textbox(label="Write Note")
            note_btn = gr.Button("Save Note")
            clear_btn = gr.Button("Clear Notes")
            download_btn = gr.Button("Download Notes")
            note_out = gr.Textbox(label="Saved Notes", lines=6)

        with gr.Tab("💡 Motivation"):
            mot_btn = gr.Button("Need Motivation?")
            mot_out = gr.Textbox(label="Motivational Quote", lines=2)

        with gr.Tab("🧠 Exam Predictor"):
            exam_btn = gr.Button("Predict Exam Questions")
            exam_out = gr.Textbox(label="Predicted Questions", lines=6)

        with gr.Tab("📊 Progress"):
            prog_btn = gr.Button("Show Progress")
            prog_out = gr.Textbox(label="Session Progress", lines=6)
            prog_bar = gr.Progress()  # ✅ Fixed: no arguments

    # 🔗 Button Functions
    login_btn.click(fn=login, inputs=[user,pwd], outputs=[hidden, login_status])
    summary_btn.click(fn=lambda f: generate_summary(f), inputs=file, outputs=summary_out)
    quiz_btn.click(fn=lambda f: generate_quiz(f), inputs=file, outputs=quiz_out)
    fc_btn.click(fn=lambda f: flashcards(f), inputs=file, outputs=fc_out)
    ans_btn.click(fn=ask_question, inputs=[file, ques], outputs=ans_out)
    note_btn.click(fn=lambda n: save_notes(n), inputs=note_in, outputs=note_out)
    clear_btn.click(fn=clear_notes, outputs=note_out)
    download_btn.click(fn=download_notes, outputs=note_out)
    mot_btn.click(fn=motivation_popup, outputs=mot_out)
    exam_btn.click(fn=exam_predictor, inputs=file, outputs=exam_out)
    prog_btn.click(fn=track_progress, outputs=prog_out) # Removed prog_bar from outputs and direct value setting

# -----------------------------
# 🌍 Deployment via LocalTunnel
# -----------------------------
import random
port = random.randint(7860, 7900)
demo.launch(server_name="0.0.0.0", server_port=port, share=False, inbrowser=False)
os.system(f"lt --port {port} > url.txt 2>&1 &")
time.sleep(3)
print("🔗 Your Public URL:")
!cat url.txt

Device set to use cpu
Device set to use cpu


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

🔗 Your Public URL:
sh: 1: lt: not found
